# **IP3_Run1**

In [1]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 74.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.2 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [2]:

import openai
import os
from IPython.display import Markdown


### **Accessing the GPT4 API**

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']
openai.api_key = my_api_key


### **Generate Mathematical Model**

In [4]:
problem = """You are the person in charge of packing in a large company. Your job is to skillfully pack items of various weights in a box with predetermined capacity. The objective is to use as few boxes as possible. There is a set of items and a set of boxes with an upper bound on the amount of boxes. Each of the items has a known weight. Each item is packed into one box and items are packed into the chosen boxes. Please formulate a mathematical optimization model for this problem.  """

In [5]:

client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the variables for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 1
)


In [6]:
print(response1.choices[0].message.content)

1. Decision Variables:
   Let's denote `x_ij` as a binary decision variable, where `x_ij` = 1 if item `i` is packed into box `j`, and `0` otherwise, for all items `i` in set I and all boxes `j` in set J.

2. Constants:
   - `W_i` represents the weight of item `i`, for each item `i` in set I.
   - `C_j` stands for the capacity of box `j`, for each box `j` in set J.
   - `B_max` is the maximum allowable number of boxes.

3. Constraints:
    - Item Weight Constraint: The total weight of items in a box should not exceed its maximum capacity.
    - Max Box Usage Constraint: The total number of boxes used should not exceed the predetermined maximum.
    - Each Item in One Box: Every item must be packed in exactly one box.

Note: These are not formulated as mathematical expressions here as the task only asks for variable formulation. Please refer to the mathematical formulation problem for detailed expressions.


### **Generate Pyomo Code**

In [7]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the objective function for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 1
)

In [8]:
print(response2.choices[0].message.content)

The objective function should minimize the number of boxes used:

min: ∑_{j ∈ J} ( ∑_{i ∈ I} x_ij ) 

Where:
- ∑_{i ∈ I} x_ij is 1 if any item is packed in box j, hence summing up over all j gives the total number of boxes used.
- The objective is to minimize this function since we want to use as few boxes as possible.


In [9]:
response3 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the constraints for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content +  response2.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 1
)

In [10]:
print(response3.choices[0].message.content)

1. Item Weight Constraint: For each box `j` in set J, the sum of the weights of items packed in the box (`i` in set I) cannot exceed the capacity of the box. Formulated as:

∑_{i ∈ I} (W_i * x_ij) <= C_j , ∀j ∈ J 

2. Max Box Usage Constraint: The total number of boxes used should not exceed the predetermined maximum. Formulated as:

∑_{j ∈ I} ∑_{i ∈ I} x_ij <= B_max 

3. Each Item in One Box: Every item must be packed in exactly one box. Formulated as:

∑_{j ∈ J} x_ij = 1 , ∀i ∈ I


In [11]:
response4 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content + response2.choices[0].message.content + response3.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 1
)

In [12]:
Markdown(response4.choices[0].message.content)

Sure, here is the corresponding Pyomo code for this optimization problem. In this code, I'm assuming that there are 3 items and 3 boxes for the sake of simplicity.

```python
from pyomo.environ import *

# Sample data
W = {1:10, 2:20, 3:30}    # Weight of items
C = {1:40, 2:50, 3:60}    # Box capacities
B_max = 2                 # Maximum number of used boxes

# Create a simple model
model = ConcreteModel()

# Define sets for items and boxes
model.I = Set(initialize=W.keys())
model.J = Set(initialize=C.keys())

# Define decision variables
model.x = Var(model.I, model.J, within=Binary)

# Define objective function
def obj_expression(model): 
    return sum(model.x[i,j] for i in model.I for j in model.J)
model.obj = Objective(rule=obj_expression, sense=minimize)

# Define item weight constraint
def weight_constraint_rule(model, j): 
    return sum(W[i]*model.x[i,j] for i in model.I) <= C[j]
model.weight_constraint = Constraint(model.J, rule=weight_constraint_rule)

# Define max box usage constraint
def box_constraint_rule(model): 
    return sum(model.x[i,j] for i in model.I for j in model.J) <= B_max
model.box_constraint = Constraint(rule=box_constraint_rule)

# Define each item in one box constraint
def single_box_rule(model, i): 
    return sum(model.x[i,j] for j in model.J) == 1
model.single_box_constraint = Constraint(model.I, rule=single_box_rule)

# Solving the model
solver = SolverFactory('cbc')
solver.solve(model)

# print the output
for i in model.I:
    for j in model.J:
        print("x[",i,",",j,"] = ", model.x[i,j].value)
```

This code formulates and solves a packing optimization problem using a binary programming approach. The solver I have employed is CBC, but you could use any other solver you prefer. Please make sure the input sizes and values are adjusted according to your actual packing problem. Our objective is to minimize the number of boxes used. We ensure this by setting the objective function as the sum of `x_ij`. Our constraints ensure that all items are packed, no box is overloaded and the total number of boxes used does not exceed the maximum. The output given by this code will indicate which item goes in which box.

### **Run the code Generated by GPT4**

In [14]:
from pyomo.environ import *

# Sample data
W = {1:10, 2:20, 3:30}    # Weight of items
C = {1:40, 2:50, 3:60}    # Box capacities
B_max = 2                 # Maximum number of used boxes

# Create a simple model
model = ConcreteModel()

# Define sets for items and boxes
model.I = Set(initialize=W.keys())
model.J = Set(initialize=C.keys())

# Define decision variables
model.x = Var(model.I, model.J, within=Binary)

# Define objective function
def obj_expression(model):
    return sum(model.x[i,j] for i in model.I for j in model.J)
model.obj = Objective(rule=obj_expression, sense=minimize)

# Define item weight constraint
def weight_constraint_rule(model, j):
    return sum(W[i]*model.x[i,j] for i in model.I) <= C[j]
model.weight_constraint = Constraint(model.J, rule=weight_constraint_rule)

# Define max box usage constraint
def box_constraint_rule(model):
    return sum(model.x[i,j] for i in model.I for j in model.J) <= B_max
model.box_constraint = Constraint(rule=box_constraint_rule)

# Define each item in one box constraint
def single_box_rule(model, i):
    return sum(model.x[i,j] for j in model.J) == 1
model.single_box_constraint = Constraint(model.I, rule=single_box_rule)

# Solving the model
solver = SolverFactory('glpk')
solver.solve(model)

# print the output
for i in model.I:
    for j in model.J:
        print("x[",i,",",j,"] = ", model.x[i,j].value)


x[ 1 , 1 ] =  None
x[ 1 , 2 ] =  None
x[ 1 , 3 ] =  None
x[ 2 , 1 ] =  None
x[ 2 , 2 ] =  None
x[ 2 , 3 ] =  None
x[ 3 , 1 ] =  None
x[ 3 , 2 ] =  None
x[ 3 , 3 ] =  None


In [18]:
from pyomo.environ import *

# Sample data
W = {1: 2,2: 2,3: 2,4: 2,5: 3,6: 3,7:4,8: 4,9: 4,10: 4,11: 4,12:4,13: 5,14:5,15: 5,16: 5,17: 5,18: 5,19: 6,20:6,21: 7,22: 7,23: 8, 24:8} # Weight of each item
C = 9    # Box capacities
B_max = 15             # Maximum number of used boxes

# Create a simple model
model = ConcreteModel()

# Define sets for items and boxes
model.I = Set(initialize=W.keys())
model.J = Set(initialize=range(1, B_max + 1))

# Define decision variables
model.x = Var(model.I, model.J, within=Binary)

# Define objective function
def obj_expression(model):
    return sum(model.x[i,j] for i in model.I for j in model.J)
model.obj = Objective(rule=obj_expression, sense=minimize)

# Define item weight constraint
def weight_constraint_rule(model, j):
    return sum(W[i]*model.x[i,j] for i in model.I) <= C
model.weight_constraint = Constraint(model.J, rule=weight_constraint_rule)

# Define max box usage constraint
def box_constraint_rule(model):
    return sum(model.x[i,j] for i in model.I for j in model.J) <= B_max
model.box_constraint = Constraint(rule=box_constraint_rule)

# Define each item in one box constraint
def single_box_rule(model, i):
    return sum(model.x[i,j] for j in model.J) == 1
model.single_box_constraint = Constraint(model.I, rule=single_box_rule)

# Solving the model
solver = SolverFactory('glpk')
solver.solve(model)

# print the output
for i in model.I:
    for j in model.J:
        print("x[",i,",",j,"] = ", model.x[i,j].value)
print(model.obj())

x[ 1 , 1 ] =  None
x[ 1 , 2 ] =  None
x[ 1 , 3 ] =  None
x[ 1 , 4 ] =  None
x[ 1 , 5 ] =  None
x[ 1 , 6 ] =  None
x[ 1 , 7 ] =  None
x[ 1 , 8 ] =  None
x[ 1 , 9 ] =  None
x[ 1 , 10 ] =  None
x[ 1 , 11 ] =  None
x[ 1 , 12 ] =  None
x[ 1 , 13 ] =  None
x[ 1 , 14 ] =  None
x[ 1 , 15 ] =  None
x[ 2 , 1 ] =  None
x[ 2 , 2 ] =  None
x[ 2 , 3 ] =  None
x[ 2 , 4 ] =  None
x[ 2 , 5 ] =  None
x[ 2 , 6 ] =  None
x[ 2 , 7 ] =  None
x[ 2 , 8 ] =  None
x[ 2 , 9 ] =  None
x[ 2 , 10 ] =  None
x[ 2 , 11 ] =  None
x[ 2 , 12 ] =  None
x[ 2 , 13 ] =  None
x[ 2 , 14 ] =  None
x[ 2 , 15 ] =  None
x[ 3 , 1 ] =  None
x[ 3 , 2 ] =  None
x[ 3 , 3 ] =  None
x[ 3 , 4 ] =  None
x[ 3 , 5 ] =  None
x[ 3 , 6 ] =  None
x[ 3 , 7 ] =  None
x[ 3 , 8 ] =  None
x[ 3 , 9 ] =  None
x[ 3 , 10 ] =  None
x[ 3 , 11 ] =  None
x[ 3 , 12 ] =  None
x[ 3 , 13 ] =  None
x[ 3 , 14 ] =  None
x[ 3 , 15 ] =  None
x[ 4 , 1 ] =  None
x[ 4 , 2 ] =  None
x[ 4 , 3 ] =  None
x[ 4 , 4 ] =  None
x[ 4 , 5 ] =  None
x[ 4 , 6 ] =  None
x[ 4 , 7 ] = 

ERROR:pyomo.common.numeric_types:evaluating object as numeric value: x[1,1]
    (object: <class 'pyomo.core.base.var.VarData'>)
No value for uninitialized NumericValue object x[1,1]


None
x[ 18 , 6 ] =  None
x[ 18 , 7 ] =  None
x[ 18 , 8 ] =  None
x[ 18 , 9 ] =  None
x[ 18 , 10 ] =  None
x[ 18 , 11 ] =  None
x[ 18 , 12 ] =  None
x[ 18 , 13 ] =  None
x[ 18 , 14 ] =  None
x[ 18 , 15 ] =  None
x[ 19 , 1 ] =  None
x[ 19 , 2 ] =  None
x[ 19 , 3 ] =  None
x[ 19 , 4 ] =  None
x[ 19 , 5 ] =  None
x[ 19 , 6 ] =  None
x[ 19 , 7 ] =  None
x[ 19 , 8 ] =  None
x[ 19 , 9 ] =  None
x[ 19 , 10 ] =  None
x[ 19 , 11 ] =  None
x[ 19 , 12 ] =  None
x[ 19 , 13 ] =  None
x[ 19 , 14 ] =  None
x[ 19 , 15 ] =  None
x[ 20 , 1 ] =  None
x[ 20 , 2 ] =  None
x[ 20 , 3 ] =  None
x[ 20 , 4 ] =  None
x[ 20 , 5 ] =  None
x[ 20 , 6 ] =  None
x[ 20 , 7 ] =  None
x[ 20 , 8 ] =  None
x[ 20 , 9 ] =  None
x[ 20 , 10 ] =  None
x[ 20 , 11 ] =  None
x[ 20 , 12 ] =  None
x[ 20 , 13 ] =  None
x[ 20 , 14 ] =  None
x[ 20 , 15 ] =  None
x[ 21 , 1 ] =  None
x[ 21 , 2 ] =  None
x[ 21 , 3 ] =  None
x[ 21 , 4 ] =  None
x[ 21 , 5 ] =  None
x[ 21 , 6 ] =  None
x[ 21 , 7 ] =  None
x[ 21 , 8 ] =  None
x[ 21 , 9 ] =  No

ValueError: No value for uninitialized NumericValue object x[1,1]

### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**